In [4]:
from sklearn.metrics import accuracy_score, f1_score
from EDA.DataReader import DataReader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [5]:
# pre-processing data
dataset = DataReader("UIT_VFSC") # UIT
# dataset = DataReader("") # dataset foody_raw
df_train = dataset.df_train
df_test = dataset.df_test

In [6]:
# Combine with sklearn svc
tf_idf = TfidfVectorizer()
data_train = tf_idf.fit_transform(df_train["corpus"])
svc = SVC(kernel='linear', C=1.0, gamma=0.1)
svc.fit(data_train, df_train["label"])
data_test = tf_idf.transform(df_test["corpus"])
y_pred = svc.predict(data_test)
accuracy_svc = accuracy_score(df_test["label"], y_pred)
print(f"Accuracy: {accuracy_svc:.4f}")

# use for multiclass (UIT)
mf1 = f1_score(df_test["label"], y_pred, average='macro')
wf1 = f1_score(df_test["label"], y_pred, average='weighted')

print(f"F1-Score: {max(mf1, wf1):.4f}")

# use for 2 class (foody)
# f1 = f1_score(df_test["label"], y_pred)
# print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8869
F1-Score: 0.8699
